## Imports and PrintSchema

In [2]:
import pyspark
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import Row, functions as F
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, FloatType
from pyspark.sql.functions import UserDefinedFunction, udf, struct

In [3]:
##http://archive.ics.uci.edu/ml/datasets/Census-Income+(KDD)

df= sqlContext.sql("SELECT * FROM census")

In [4]:
df.printSchema()

root
-- _c0: integer (nullable = true)
-- _c1: string (nullable = true)
-- _c2: double (nullable = true)
-- _c3: double (nullable = true)
-- _c4: string (nullable = true)
-- _c5: double (nullable = true)
-- _c6: string (nullable = true)
-- _c7: string (nullable = true)
-- _c8: string (nullable = true)
-- _c9: string (nullable = true)
-- _c10: string (nullable = true)
-- _c11: string (nullable = true)
-- _c12: string (nullable = true)
-- _c13: string (nullable = true)
-- _c14: string (nullable = true)
-- _c15: string (nullable = true)
-- _c16: double (nullable = true)
-- _c17: double (nullable = true)
-- _c18: double (nullable = true)
-- _c19: string (nullable = true)
-- _c20: string (nullable = true)
-- _c21: string (nullable = true)
-- _c22: string (nullable = true)
-- _c23: string (nullable = true)
-- _c24: double (nullable = true)
-- _c25: string (nullable = true)
-- _c26: string (nullable = true)
-- _c27: string (nullable = true)
-- _c28: string (nullable = true)
-- _c29: string (nullable = true)
-- _c30: double (nullable = true)
-- _c31: string (nullable = true)
-- _c32: string (nullable = true)
-- _c33: string (nullable = true)
-- _c34: string (nullable = true)
-- _c35: string (nullable = true)
-- _c36: double (nullable = true)
-- _c37: string (nullable = true)
-- _c38: double (nullable = true)
-- _c39: double (nullable = true)
-- _c40: double (nullable = true)
-- _c41: string (nullable = true)

In [5]:
display(df.head(3))

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,_c26,_c27,_c28,_c29,_c30,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,_c40,_c41
73,Not in universe,0.0,0.0,High school graduate,0.0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,0.0,95.0,- 50000.
58,Self-employed-not incorporated,4.0,34.0,Some college but no degree,0.0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,52.0,94.0,- 50000.
18,Not in universe,0.0,0.0,10th grade,0.0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0.0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0.0,Not in universe,2.0,0.0,95.0,- 50000.


## Display DF and separating Numerical From Categorical

In [7]:
#df= df.withColumnRenamed('_c0', 'age').withColumnRenamed('_c41', 'Target')
display(df.head(3))

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,_c26,_c27,_c28,_c29,_c30,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,_c40,_c41
73,Not in universe,0.0,0.0,High school graduate,0.0,Not in universe,Widowed,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Other Rel 18+ ever marr not in subfamily,Other relative of householder,1700.09,?,?,?,Not in universe under 1 year old,?,0.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,0.0,95.0,- 50000.
58,Self-employed-not incorporated,4.0,34.0,Some college but no degree,0.0,Not in universe,Divorced,Construction,Precision production craft & repair,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Head of household,South,Arkansas,Householder,Householder,1053.55,MSA to MSA,Same county,Same county,No,Yes,1.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,52.0,94.0,- 50000.
18,Not in universe,0.0,0.0,10th grade,0.0,High school,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child 18+ never marr Not in a subfamily,Child 18 or older,991.95,?,?,?,Not in universe under 1 year old,?,0.0,Not in universe,Vietnam,Vietnam,Vietnam,Foreign born- Not a citizen of U S,0.0,Not in universe,2.0,0.0,95.0,- 50000.


In [8]:
print("DataFrame has got {} rows and {} columns".format(df.count(), len(df.columns)))

DataFrame has got 199523 rows and 42 columns

In [11]:
""" #https://www2.1010data.com/documentationcenter/prod/Tutorials/MachineLearningExamples/CensusIncomeDataSet.html#reference_vn5_sxt_z4__input_variables

mapping_cols= {  
              '_c0': 'age',
             '_c1': 'class_worker',
             '_c2': 'industry_code',
             '_c3': 'occupation_code',
             '_c4': 'education',
             '_c5': 'wage_per_hour',
             '_c6': 'hs_college',
              '_c7': 'marital_status',
             '_c8': 'major_ind_code',
             '_c9': 'major_occ_code',
             '_c10': 'race',
             '_c11': 'hisp_origin',
             '_c12': 'gender',
             '_c13': 'labor_union_member',
             '_c14': 'unemployment_reason',
             '_c15': 'full_or_part_emp',
             '_c16': 'capital_gains',
             '_c17': 'capital_losses',
             '_c18': 'stock_dividends',
             '_c19': 'tax_filer_stat',
             '_c20': 'region_prev_res',
             '_c21': 'state_prev_res',
             '_c22': 'family_status',
             '_c23': 'family_summary',
             '_c25': 'mig_chg_msa',
             '_c26': 'mig_chg_reg',
             '_c27': 'mig_move_reg',
             '_c28': 'mig_same',
             '_c29': 'mig_prev_sunbelt',
             '_c30': 'num_employees',
             '_c31': 'familiars_U18',
             '_c32': 'country_father',
             '_c33': 'country_mother',
             '_c34': 'country_self',
             '_c35': 'citizenship',
             '_c36': 'own_business',
             '_c37': 'vet_question',
             '_c39': 'weeks_worked',
             '_c40': 'survey_year',
             '_c41': 'Target'}

In [12]:
for col_name in mapping_cols:
  df= df.withColumnRenamed(col_name, mapping_cols[col_name])
display(df.sample(withReplacement= False, fraction=0.1).head(10))

age,class_worker,industry_code,occupation_code,education,wage_per_hour,hs_college,marital_status,major_ind_code,major_occ_code,race,hisp_origin,gender,labor_union_member,unemployment_reason,full_or_part_emp,capital_gains,capital_losses,stock_dividends,tax_filer_stat,region_prev_res,state_prev_res,family_status,family_summary,_c24,mig_chg_msa,mig_chg_reg,mig_move_reg,mig_same,mig_prev_sunbelt,num_employees,familiars_U18,country_father,country_mother,country_self,citizenship,own_business,vet_question,_c38,weeks_worked,survey_year,Target
10,Not in universe,0.0,0.0,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,1069.16,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,94.0,- 50000.
28,Private,4.0,40.0,High school graduate,0.0,Not in universe,Never married,Construction,Handlers equip cleaners etc,White,All other,Female,Not in universe,Job loser - on layoff,Unemployed full-time,0.0,0.0,0.0,Single,Not in universe,Not in universe,Secondary individual,Nonrelative of householder,898.83,?,?,?,Not in universe under 1 year old,?,4.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,30.0,95.0,- 50000.
47,Private,39.0,36.0,Bachelors degree(BA AB BS),0.0,Not in universe,Never married,Personal services except private HH,Machine operators assmblrs & inspctrs,White,All other,Female,Not in universe,Not in universe,Full-time schedules,0.0,0.0,0.0,Single,Not in universe,Not in universe,Nonfamily householder,Householder,404.9,?,?,?,Not in universe under 1 year old,?,6.0,Not in universe,Germany,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,52.0,95.0,- 50000.
63,Private,19.0,35.0,Less than 1st grade,0.0,Not in universe,Married-civilian spouse present,Manufacturing-nondurable goods,Precision production craft & repair,Other,Mexican (Mexicano),Male,Not in universe,Job loser - on layoff,Unemployed full-time,0.0,0.0,0.0,Joint both under 65,Not in universe,Not in universe,Householder,Householder,711.15,?,?,?,Not in universe under 1 year old,?,6.0,Not in universe,Mexico,Mexico,Mexico,Foreign born- Not a citizen of U S,0.0,Not in universe,2.0,15.0,95.0,- 50000.
84,Not in universe,0.0,0.0,High school graduate,0.0,Not in universe,Married-civilian spouse present,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Not in labor force,0.0,0.0,0.0,Joint both 65+,Not in universe,Not in universe,Spouse of householder,Spouse of householder,3494.72,?,?,?,Not in universe under 1 year old,?,0.0,Not in universe,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,2.0,0.0,95.0,- 50000.
5,Not in universe,0.0,0.0,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,West,Alaska,Child <18 never marr not in subfamily,Child under 18 never married,235.92,MSA to MSA,Same county,Same county,No,No,0.0,Mother only present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,94.0,- 50000.
35,Not in universe,0.0,0.0,High school graduate,0.0,Not in universe,Married-civilian spouse present,Not in universe or children,Not in universe,White,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Joint both under 65,Not in universe,Not in universe,Spouse of householder,Spouse of householder,1669.47,Nonmover,Nonmover,Nonmover,Yes,Not in universe,0.0,Not in universe,United-States,United-States,United-States,Native- Born in the Uni

###Dropping duplicates and Handling Missing Values

In [14]:
def title_ticks_ecc(title, xlabel, ylabel, new_fig= True , figsize= (10,6)):
    if new_fig== True:
        plt.figure(figsize=figsize)
        
    plt.title(title, color='red', fontsize= 18, pad= 13)
    plt.xlabel(xlabel, color='red', fontsize= 14)
    plt.ylabel(ylabel, color='red', fontsize= 14)
    plt.xticks(fontsize=12, color='blue')
    plt.yticks(fontsize=12, color='blue')  

In [15]:
print('There are {} duplicates rows which are going to be eliminated'.format(df.count()-  df.drop_duplicates().count()))
df= df.drop_duplicates()
print('There are {} rows out with at least one NaN (missing value)'.format(df.count()- df.na.drop(how='any').count()))

There are 3229 duplicates rows which are going to be eliminated
There are 0 rows out with at least one NaN (missing value)

In [16]:
counts = df.agg(*(countDistinct(col(c)).alias(c) for c in df.columns)) #pd.Series(counts.rdd.flatMap(lambda x: x).histogram(42)[0]).sort_values(ascending= False)  
for col in mapping_cols:
  counts= counts.withColumnRenamed(col, mapping_cols[col])
display(counts)

age,class_worker,industry_code,occupation_code,education,wage_per_hour,hs_college,marital_status,major_ind_code,major_occ_code,race,hisp_origin,gender,labor_union_member,unemployment_reason,full_or_part_emp,capital_gains,capital_losses,stock_dividends,tax_filer_stat,region_prev_res,state_prev_res,family_status,family_summary,_c24,mig_chg_msa,mig_chg_reg,mig_move_reg,mig_same,mig_prev_sunbelt,num_employees,familiars_U18,country_father,country_mother,country_self,citizenship,own_business,vet_question,_c38,weeks_worked,survey_year,Target
91,9,52,47,17,1240,3,7,24,15,5,10,2,3,6,8,132,113,1478,6,6,51,38,8,99800,10,9,10,3,4,7,5,43,43,43,5,3,3,3,53,2,2


In [17]:
dict_counts= counts.collect()[0].asDict()
series_counts= pd.Series(dict_counts).sort_values(ascending= False)
df.select(F.countDistinct('age')).show()

+-------------------+
count(DISTINCT age)|
+-------------------+
 91|
+-------------------+

In [18]:
display(df.sample(withReplacement= False, fraction=0.1).head(3))

age,class_worker,industry_code,occupation_code,education,wage_per_hour,hs_college,marital_status,major_ind_code,major_occ_code,race,hisp_origin,gender,labor_union_member,unemployment_reason,full_or_part_emp,capital_gains,capital_losses,stock_dividends,tax_filer_stat,region_prev_res,state_prev_res,family_status,family_summary,_c24,mig_chg_msa,mig_chg_reg,mig_move_reg,mig_same,mig_prev_sunbelt,num_employees,familiars_U18,country_father,country_mother,country_self,citizenship,own_business,vet_question,_c38,weeks_worked,survey_year,Target
3,Not in universe,0.0,0.0,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,Asian or Pacific Islander,All other,Female,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Grandchild <18 never marr child of subfamily RP,Other relative of householder,982.2,?,?,?,Not in universe under 1 year old,?,0.0,Mother only present,?,Philippines,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,95.0,- 50000.
0,Not in universe,0.0,0.0,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,White,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,Not in universe,Not in universe,Child <18 never marr not in subfamily,Child under 18 never married,4605.5,?,?,?,Not in universe under 1 year old,?,0.0,Both parents present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,95.0,- 50000.
13,Not in universe,0.0,0.0,Children,0.0,Not in universe,Never married,Not in universe or children,Not in universe,Black,All other,Male,Not in universe,Not in universe,Children or Armed Forces,0.0,0.0,0.0,Nonfiler,West,Arizona,Child <18 never marr not in subfamily,Child under 18 never married,2151.01,MSA to MSA,Same county,Same county,No,Yes,0.0,Mother only present,United-States,United-States,United-States,Native- Born in the United States,0.0,Not in universe,0.0,0.0,94.0,- 50000.


In [19]:
pandas_df= df.toPandas()
pandas_df.sample(3)
categorical_feats= list(pandas_df.select_dtypes(include= object).columns)
categorical_feats.remove('Target')
numerical_feats= list(pandas_df.select_dtypes(include= 'number').columns)

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [20]:
diz_nan= {}
for col in categorical_feats:
  diz_nan[col]= df.filter(df[col]== ' ?').count()
print(diz_nan)
missing_values= pd.Series(diz_nan).sort_values(ascending= False)
missing_values= missing_values[missing_values>0]
title_ticks_ecc('Number of NaNs out of {} samples'.format(df.count()),'Variable Name','N. Missing')
display(missing_values.plot(kind='bar'))

In [21]:
missing_values

Out[33]: mig_chg_reg 93756
mig_prev_sunbelt 93756
mig_move_reg 93756
mig_chg_msa 93756
country_father 3252
country_mother 2961
country_self 1606
state_prev_res 707
dtype: int64

In [ ]:
num_missing_udf  = udf(lambda row: len([x for x in row if x == ' ?']), IntegerType())
new_df= df.withColumn('Count_missing_row', num_missing_udf(struct([df[x] for x in df.columns])))
#display(new_df)

#### Chosing a Threshold of Nans, I drop those rows having a Number of Missing Values higher than that. Then I will fill the others.

In [24]:
new_df.select('Count_missing_row').describe().toPandas().set_index('summary').transpose()

summary,count,mean,stddev,min,max
Count_missing_row,196294,2.0834360703842196,2.0460276581734336,0,7


In [25]:
display(new_df.groupBy('Count_missing_row').agg({'Count_missing_row': 'count'})).orderBy('Count_missing_row')

Count_missing_row,count(Count_missing_row)
1,2215
6,1195
3,1335
5,1601
4,93770
7,1463
2,1125
0,93590


In [26]:
threshold= 5
df= new_df.filter(new_df['Count_missing_row'] < threshold).drop('Count_missing_row')
print("New DataFrame has got {} rows and {} columns".format(df.count(), len(df.columns)))

New DataFrame has got 192035 rows and 42 columns

In [27]:
display(df)

In [28]:
diz_nan= {}
for col in categorical_feats:
  diz_nan[col]= df.filter(df[col]== ' ?').count()
missing_values= pd.Series(diz_nan).sort_values(ascending= False)
missing_values= missing_values[missing_values>0]
title_ticks_ecc('(Final) Number of NaNs out of {} Rows'.format(df.count()),'Variable Name','N. Missing')
display(missing_values.plot(kind='bar'))

### Transform Target in Binary and check its distribution

In [30]:
from pyspark.sql.functions import UserDefinedFunction
create_target = UserDefinedFunction(lambda x: int(0) if x== ' - 50000.' else int(1), IntegerType() ) 
df = df.withColumn("Target", create_target('Target'))
df.select('Target').collect()[:5]

Out[22]: [Row(Target=0), Row(Target=0), Row(Target=0), Row(Target=0), Row(Target=0)]

In [31]:
df.select(F.countDistinct('Target')).show()   #check ok

+----------------------+
count(DISTINCT Target)|
+----------------------+
 2|
+----------------------+

In [32]:
fig= plt.figure()
target_series= df.select('Target').toPandas()
sns.countplot(target_series.squeeze())
display(fig)
##display(count_targets.toPandas().plot(kind= 'bar'))

In [33]:
from matplotlib.pyplot import pie, axis, show

sums = target_series.squeeze().value_counts()
sums.index= sums.index.map({1: 'Class 1', 0: 'Class 0'})
#plt.figure(figsize=(20,10))

fig1, ax1 = plt.subplots(figsize= (7,7))
plt.title("Distribution Binary Target", fontsize= 15, color='orange')
_, _, autotexts = pie(sums,labels= sums.index, center= (1,1), autopct='%1.1f%%', pctdistance=0.8,
        shadow=True, startangle=90, textprops={'fontsize': 22, 'color':"black", 'weight':'bold'}, labeldistance= 1.0)

for autotext in autotexts:
    autotext.set_color('white')
    
"""ax1.pie(sums, labels=sums.index, center= (1,1), autopct='%1.1f%%',
        shadow=True, startangle=90, textprops={'fontsize': 20, 'color':"black"}, labeldistance= 1.0)"""
ax1.axis('equal')
ax1.legend(loc="lower left",
          bbox_to_anchor=(1, 0, 0.5, 1), fontsize=20)
plt.setp(autotexts, weight="bold")
display(plt.show())
plt.close()

In [34]:
new_categorical= categorical_feats + list (series_counts.loc[numerical_feats].sort_values(ascending= False).iloc[7:].index)
new_numerical= [col for col in numerical_feats if col not in new_categorical]
frequency_class1= 0.063

In [35]:
df.select(categorical_feats).printSchema()

root
-- class_worker: string (nullable = true)
-- education: string (nullable = true)
-- hs_college: string (nullable = true)
-- marital_status: string (nullable = true)
-- major_ind_code: string (nullable = true)
-- major_occ_code: string (nullable = true)
-- race: string (nullable = true)
-- hisp_origin: string (nullable = true)
-- gender: string (nullable = true)
-- labor_union_member: string (nullable = true)
-- unemployment_reason: string (nullable = true)
-- full_or_part_emp: string (nullable = true)
-- tax_filer_stat: string (nullable = true)
-- region_prev_res: string (nullable = true)
-- state_prev_res: string (nullable = true)
-- family_status: string (nullable = true)
-- family_summary: string (nullable = true)
-- mig_chg_msa: string (nullable = true)
-- mig_chg_reg: string (nullable = true)
-- mig_move_reg: string (nullable = true)
-- mig_same: string (nullable = true)
-- mig_prev_sunbelt: string (nullable = true)
-- familiars_U18: string (nullable = true)
-- country_father: string (nullable = true)
-- country_mother: string (nullable = true)
-- country_self: string (nullable = true)
-- citizenship: string (nullable = true)
-- vet_question: string (nullable = true)

##### Transforming all the categorical features into Lower case

In [37]:
lower_category = UserDefinedFunction(lambda x: x.lower(), StringType()) 

for cat_col in categorical_feats:
  df= df.withColumn(cat_col, lower_category(cat_col))

## Explorative Analysis

In [39]:
print('There are initially {} categorical columns and {} numerical'.format(len(categorical_feats), len(numerical_feats)))

In [40]:
####### There is no too much 'dispersion' withing the categorical variables'. Maximum dispersion is one different value for each row, like an ID; a feature like that would surely not be important#########
title_ticks_ecc('Categorical Variables Cardinalities', 'Category Name', 'N° Classes', figsize= (10,6))  
display(series_counts.loc[categorical_feats].sort_values(ascending= False).plot(kind= 'bar'))

In [41]:
title_ticks_ecc('N° distinct values Numerical Feats', 'Variable Name', 'N. Values', figsize= (10, 6))
display(series_counts.loc[numerical_feats].sort_values(ascending= False).plot(kind= 'bar'))

In [42]:
title_ticks_ecc('N° distinct values Numerical Feats', 'Variable Name', 'N. Values', figsize= (10, 6))
display(series_counts.loc[numerical_feats].sort_values(ascending= False).iloc[1:].plot(kind= 'bar'))

In [43]:
series_counts.loc[numerical_feats].sort_values(ascending= False)

Out[42]: _c24 99800
stock_dividends 1478
wage_per_hour 1240
capital_gains 132
capital_losses 113
age 91
weeks_worked 53
industry_code 52
occupation_code 47
num_employees 7
_c38 3
own_business 3
survey_year 2
dtype: int64

In [44]:
display(df.select(categorical_feats).head(3))

class_worker,education,hs_college,marital_status,major_ind_code,major_occ_code,race,hisp_origin,gender,labor_union_member,unemployment_reason,full_or_part_emp,tax_filer_stat,region_prev_res,state_prev_res,family_status,family_summary,mig_chg_msa,mig_chg_reg,mig_move_reg,mig_same,mig_prev_sunbelt,familiars_U18,country_father,country_mother,country_self,citizenship,vet_question
private,associates degree-occup /vocational,not in universe,married-civilian spouse present,manufacturing-nondurable goods,sales,white,all other,male,not in universe,not in universe,pt for non-econ reasons usually ft,joint both 65+,not in universe,not in universe,householder,householder,?,?,?,not in universe under 1 year old,?,not in universe,hungary,england,united-states,native- born in the united states,not in universe
not in universe,children,not in universe,never married,not in universe or children,not in universe,black,all other,female,not in universe,not in universe,children or armed forces,nonfiler,not in universe,not in universe,child <18 never marr not in subfamily,child under 18 never married,nonmover,nonmover,nonmover,yes,not in universe,both parents present,united-states,united-states,united-states,native- born in the united states,not in universe
not in universe,high school graduate,not in universe,married-civilian spouse present,not in universe or children,not in universe,white,all other,female,not in universe,not in universe,children or armed forces,joint both 65+,not in universe,not in universe,spouse of householder,spouse of householder,nonmover,nonmover,nonmover,yes,not in universe,not in universe,poland,poland,united-states,native- born in the united states,not in universe


## Modyfing Schema

In [46]:
df.select(numerical_feats).printSchema()
df= df.withColumn('_c24', df['_c24'].cast('float'))

for col in numerical_feats:
  if col!= '_c24':
    df= df.withColumn(col, df[col].cast('int'))
    
print('New Schema')
df.select(numerical_feats).printSchema()

root
-- age: integer (nullable = true)
-- industry_code: double (nullable = true)
-- occupation_code: double (nullable = true)
-- wage_per_hour: double (nullable = true)
-- capital_gains: double (nullable = true)
-- capital_losses: double (nullable = true)
-- stock_dividends: double (nullable = true)
-- _c24: double (nullable = true)
-- num_employees: double (nullable = true)
-- own_business: double (nullable = true)
-- _c38: double (nullable = true)
-- weeks_worked: double (nullable = true)
-- survey_year: double (nullable = true)

New Schema
root
-- age: integer (nullable = true)
-- industry_code: integer (nullable = true)
-- occupation_code: integer (nullable = true)
-- wage_per_hour: integer (nullable = true)
-- capital_gains: integer (nullable = true)
-- capital_losses: integer (nullable = true)
-- stock_dividends: integer (nullable = true)
-- _c24: float (nullable = true)
-- num_employees: integer (nullable = true)
-- own_business: integer (nullable = true)
-- _c38: integer (nullable = true)
-- weeks_worked: integer (nullable = true)
-- survey_year: integer (nullable = true)

In [47]:
"""schemaNum = numerical_feats #è una lista di stringhe, ogni stringa diventerà nome del campo

fields = [StructField(schemaNum[0], IntegerType(), True)]+ [StructField(field_name, FloatType(), True) for field_name in schemaNum][1:]  #field è una lista di campi
schema = StructType(fields)
rdd_vuoto= sc.emptyRDD()
# Apply the schema to the RDD.
df_solo_schema = spark.createDataFrame(rdd_vuoto, schema)

df.select(numerical_feats).printSchema()

df_num= spark.createDataFrame(df.select(numerical_feats).rdd.collect(), schema)"""

Out[37]: 'schemaNum = numerical_feats #è una lista di stringhe, ogni stringa diventerà nome del campo\n\nfields = [StructField(schemaNum[0], IntegerType(), True)]+ [StructField(field_name, FloatType(), True) for field_name in schemaNum][1:] #field è una lista di campi\nschema = StructType(fields)\nrdd_vuoto= sc.emptyRDD()\n# Apply the schema to the RDD.\ndf_solo_schema = spark.createDataFrame(rdd_vuoto, schema)\n\ndf.select(numerical_feats).printSchema()\n\ndf_num= spark.createDataFrame(df.select(numerical_feats).rdd.collect(), schema)'

In [48]:
df_numerical= df.select(numerical_feats)
print('Number of Missing Values in Numerical columns: ', df_numerical.na.drop(how='any').count())  #No missing values.
df_numerical.show(3)

Number of Missing Values in Numerical columns: 192035
+---+-------------+---------------+-------------+-------------+--------------+---------------+-------+-------------+------------+----+------------+-----------+
age|industry_code|occupation_code|wage_per_hour|capital_gains|capital_losses|stock_dividends| _c24|num_employees|own_business|_c38|weeks_worked|survey_year|
+---+-------------+---------------+-------------+-------------+--------------+---------------+-------+-------------+------------+----+------------+-----------+
 53| 0| 0| 0| 0| 0| 0|1920.08| 0| 0| 2| 0| 95|
 58| 4| 34| 0| 0| 0| 0|1053.55| 1| 0| 2| 52| 94|
 28| 47| 28| 1175| 0| 0| 50| 371.59| 1| 0| 2| 52| 94|
+---+-------------+---------------+-------------+-------------+--------------+---------------+-------+-------------+------------+----+------------+-----------+
only showing top 3 rows

In [49]:
display(counts.select(numerical_feats))

age,industry_code,occupation_code,wage_per_hour,capital_gains,capital_losses,stock_dividends,_c24,num_employees,own_business,_c38,weeks_worked,survey_year
91,52,47,1240,132,113,1478,99800,7,3,3,53,2


##Since many operations are going to be done on df, it is useful to persist it, so that spark does not have to rebuild it everytime

In [51]:
df.persist()

Out[34]: DataFrame[age: int, class_worker: string, industry_code: int, occupation_code: int, education: string, wage_per_hour: int, hs_college: string, marital_status: string, major_ind_code: string, major_occ_code: string, race: string, hisp_origin: string, gender: string, labor_union_member: string, unemployment_reason: string, full_or_part_emp: string, capital_gains: int, capital_losses: int, stock_dividends: int, tax_filer_stat: string, region_prev_res: string, state_prev_res: string, family_status: string, family_summary: string, _c24: float, mig_chg_msa: string, mig_chg_reg: string, mig_move_reg: string, mig_same: string, mig_prev_sunbelt: string, num_employees: int, familiars_U18: string, country_father: string, country_mother: string, country_self: string, citizenship: string, own_business: int, vet_question: string, _c38: int, weeks_worked: int, survey_year: int, Target: int]

### Numerical Features plots

In [53]:
num_pandas= df.select(numerical_feats + ['Target']).toPandas()

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [54]:
fig= plt.figure(figsize= (30, 10))
n_cols= 5
n_rows= 3

for i,feat in enumerate(numerical_feats):
  plt.subplot(5,3, i+1)
  plt.title('Distribution {}'.format(feat.upper()), color='red', fontsize= 12)
  col_name= numerical_feats[i]
  num_pandas[col_name].hist()
  plt.xlim(num_pandas[col_name].min()- 0.2, num_pandas[col_name].max()+ 0.2)
plt.tight_layout()
display(fig)

In [55]:
"""fig= plt.figure(figsize= (30, 10))
n_cols= 5
n_rows= 3

for i in range(len(numerical_feats)):
  plt.subplot(5,3, i+1)
  col_name= numerical_feats[i]
  sns.distplot(df_num.select([col_name]).collect())

# set ticks and tick labels
display(fig)"""

Out[75]: 'fig= plt.figure(figsize= (30, 10))\nn_cols= 5\nn_rows= 3\n\nfor i in range(len(numerical_feats)):\n plt.subplot(5,3, i+1)\n col_name= numerical_feats[i]\n sns.distplot(df_num.select([col_name]).collect())\n\n# set ticks and tick labels\ndisplay(fig)'

In [56]:
num_pandas= num_pandas[['age', 'industry_code', 'occupation_code', '_c24', 'num_employees', 'Target']]
num_pandas.head()

,age,industry_code,occupation_code,_c24,num_employees,Target
0,34,4,37,1146.790039,6,0
1,31,5,39,636.849976,2,0
2,15,0,0,2867.129883,0,0
3,31,39,3,441.160004,2,0
4,50,40,29,920.070007,3,0


In [57]:
fig= plt.figure(figsize= (30, 10))
n_cols= 5
n_rows= 2

for i, feat in enumerate(list(num_pandas.columns[:-1])):
  plt.subplot(2,3, i+1)
  col_name= num_pandas.columns[i]
  plt.title(col_name, fontsize= 13, color= 'red')
  sns.boxplot(df.select([col_name]).collect(), orient= 'vertical')

# set ticks and tick labels
display(fig)

In [58]:
def distplot_with_hue(data=None, x=None, hue=None, row=None, col=None, legend=True, plot_type= sns.distplot, **kwargs):
    _, bins = np.histogram(data[x].dropna())
    g = sns.FacetGrid(data, hue=hue, row=row, col=col)
    g.map(plot_type, x, **kwargs)
    if legend and (hue is not None) and (hue not in [x, row, col]):
        g.add_legend(title=hue) 

In [59]:
display(distplot_with_hue(data= num_pandas, x='occupation_code', hue='Target', hist=False))

In [60]:
plt.figure(figsize=(10,10))
sns.pairplot(data= num_pandas, hue= 'Target', vars= ['age', '_c24', 'num_employees']) 
display(plt.show())

In [61]:
plt.figure(figsize=(10,10))
sns.pairplot(data= num_pandas, hue= 'Target', vars= ['age', '_c24', 'num_employees', 'industry_code', 'occupation_code']) 
display(plt.show())

In [62]:
plt.figure(figsize=(10,7))
title_ticks_ecc('Correlation between features', '', '', new_fig= False)
sns.heatmap(num_pandas.corr(), cmap= 'coolwarm', annot= True)
display(plt.show())

In [63]:
plt.figure(figsize=(10,5))
num_pandas.drop(['Target', '_c24'], axis=1).plot(kind= 'box', figsize= (25,8))
title_ticks_ecc('BoxPlots', '', '', figsize=(30, 10), new_fig= False)
plt.ylim(-1, 91)
display(plt.show())

In [64]:
plt.figure(figsize=(10,5))
sns.boxplot(df.select('_c24').collect(), orient= 'vertical')
title_ticks_ecc('BoxPlot _c24', '', '', figsize=(30, 10), new_fig= False)
plt.ylim(-50, 5000)
display(plt.show())

## Statistics on Categorical Variables

In [66]:
title_ticks_ecc('Categorical Variables Cardinalities', 'Category Name', 'N° Classes', figsize= (10,6))
display(series_counts.loc[new_categorical].sort_values(ascending= False).plot(kind= 'bar'))

In [67]:
series_counts.loc[new_categorical].sort_values(ascending= False).iloc[10:]

Out[28]: mig_chg_msa 10
hisp_origin 10
mig_move_reg 10
class_worker 9
mig_chg_reg 9
full_or_part_emp 8
family_summary 8
num_employees 7
marital_status 7
region_prev_res 6
tax_filer_stat 6
unemployment_reason 6
familiars_U18 5
citizenship 5
race 5
mig_prev_sunbelt 4
own_business 3
_c38 3
labor_union_member 3
mig_same 3
vet_question 3
hs_college 3
gender 2
survey_year 2
dtype: int64

In [68]:
plt.figure(figsize= (15,5))
title_ticks_ecc('Countplot for Num_employees', '', '', new_fig= False)
display(sns.countplot("num_employees", hue= 'Target', data= df.select("num_employees", "Target").toPandas()))

In [69]:
grouped_num_emp= df.select("num_employees", "Target").groupBy("num_employees").agg({'Target' :'mean', 'num_employees': 'count'}).withColumnRenamed('avg(Target)','Mean_Target').withColumnRenamed(
                                                                                                                                          'count(num_employees)','N_samples')
grouped_num_emp= grouped_num_emp.join(df.select("num_employees", "Target").groupBy("num_employees").agg({'Target' :'std'}).withColumnRenamed('stddev(Target)','Std_Target'), on='num_employees', how='left')
grouped_num_emp= grouped_num_emp.orderBy('N_samples', ascending= False)

display(grouped_num_emp)

num_employees,Mean_Target,N_samples,Std_Target
0,0.0062370752098377695,90427,0.0787289187454228
6,0.14900265029990237,35845,0.3560960517600929
1,0.08951135459927542,22634,0.2854867298015117
4,0.10069444444444445,14112,0.300934362680068
3,0.09126743480897513,13192,0.28799996110694887
2,0.08197879858657244,9905,0.27434626650391797
5,0.10962837837837838,5920,0.3124523771606084


In [70]:
plt.figure(figsize= (20, 15))
grouped_num_emp.toPandas().set_index('num_employees')[['Mean_Target', 'Std_Target']].plot(kind='bar')
title_ticks_ecc('Employee target', 'Num_employees', '', new_fig= False)
display(plt.show())

In [71]:
grouped_country= df.select("country_self", "Target").groupBy("country_self").agg({'Target' :'mean', 'country_self': 'count'}).withColumnRenamed('avg(Target)','Mean_Target').withColumnRenamed(
                                                                                                                                          'count(country_self)','N_samples')
grouped_country= grouped_country.join(df.select("country_self", "Target").groupBy("country_self").agg({'Target' :'std'}).withColumnRenamed('stddev(Target)','Std_Target'), on='country_self', how='left')
grouped_country= grouped_country.orderBy('N_samples', ascending= False)

display(grouped_country)

country_self,Mean_Target,N_samples,Std_Target
united-states,0.0639528443244693,171517,0.2446696283754893
mexico,0.012519561815336464,5751,0.11119789765708607
?,0.08468244084682441,1606,0.2784952759375307
puerto-rico,0.02158273381294964,1390,0.14536891820416742
philippines,0.09263657957244656,842,0.29009479526836474
germany,0.08443908323281062,829,0.2782130352147608
cuba,0.07178841309823678,794,0.25829995284767354
el-salvador,0.018950437317784258,686,0.13644947362970325
dominican-republic,0.01608187134502924,684,0.12588253265691046
canada,0.09865470852017937,669,0.2984209672745601


In [72]:
############ More Compact Version #############

display(df.select("country_self", "Target").groupBy("country_self").agg((F.mean('Target')).alias('Mean_Target'), F.count('country_self').alias('N_samples'), F.stddev('Target').alias('Std_Target') 
                                                                       ).orderBy('N_samples', ascending= False))

country_self,Mean_Target,N_samples,Std_Target
united-states,0.0639528443244693,171517,0.2446696283754893
mexico,0.012519561815336464,5751,0.11119789765708607
?,0.08468244084682441,1606,0.2784952759375307
puerto-rico,0.02158273381294964,1390,0.14536891820416742
philippines,0.09263657957244656,842,0.29009479526836474
germany,0.08443908323281062,829,0.2782130352147608
cuba,0.07178841309823678,794,0.25829995284767354
el-salvador,0.018950437317784258,686,0.13644947362970325
dominican-republic,0.01608187134502924,684,0.12588253265691046
canada,0.09865470852017937,669,0.2984209672745601


In [73]:
#### factor= mean/0.062
#### scaling= (factor* 100) /(factor+1)
n_to_show= 5

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
df_pandas= grouped_country.toPandas().iloc[:n_to_show].set_index('country_self')[['Mean_Target', 'Std_Target']]

df_pandas.plot(ax=ax[0], title='Original', kind= 'bar')
ax[0].set_ylim(0,1) 

df_pandas['Mean_Target'] = ((100*df_pandas['Mean_Target']/ frequency_class1)/ (1+ df_pandas['Mean_Target']/ frequency_class1))/100
df_pandas.plot(ax=ax[1], title='Rescaled', kind= 'bar')
ax[1].set_ylim(0,1)

"""df_pandas.plot(ax=ax[1][0],
        title='Bottom left', kind= 'bar') #deve avere subplots( 2,2)"""

plt.tight_layout()
display(fig)

In [74]:
prova= grouped_country.toPandas().set_index('country_self')
prova['N_samples']= (prova['N_samples']/df.count()).cumsum()
prova= prova[prova['N_samples'] < 0.95]
prova.rename({'N_samples': 'Cum_%_samples'}, axis= 1, inplace= True)
prova

,Mean_Target,Cum_%_samples,Std_Target
country_self,,,
united-states,0.063953,0.893155,0.244670
mexico,0.012520,0.923103,0.111198
?,0.084682,0.931466,0.278495
puerto-rico,0.021583,0.938704,0.145369
philippines,0.092637,0.943088,0.290095
germany,0.084439,0.947405,0.278213


In [75]:
grouped_family= df.select("family_status", "Target").groupBy("family_status").agg((F.mean('Target')).alias('Mean_Target'), F.count('family_status').alias('N_samples'), F.stddev('Target').alias('Std_Target') 
                                                                       ).orderBy('N_samples', ascending= False)
prova= grouped_family.toPandas().set_index('family_status')
prova['N_samples']= (prova['N_samples']/df.count()).cumsum()
prova= prova[prova['N_samples'] < 0.85]
prova.rename({'N_samples': 'Cum_%_samples'}, axis= 1, inplace= True)
prova

,Mean_Target,Cum_%_samples,Std_Target
family_status,,,
householder,0.146918,0.270758,0.354028
child <18 never marr not in subfamily,0.000043,0.512729,0.006561
spouse of householder,0.054118,0.724613,0.226253
nonfamily householder,0.081085,0.836551,0.272972


## Compact Statistics on Categorical Variables (all in one plot)

In [77]:
tot_cat_feats= len(series_counts.loc[new_categorical].sort_values(ascending= False).iloc[:])
n_cols= 5
n_rows= int(np.ceil(tot_cat_feats/n_cols))
n_to_show= 4
fig, ax = plt.subplots(n_rows, n_cols, figsize=(30,40))
threshold_samples= 0.8
proportion_class1= float(0.063)

iteraz= 0
for i in range(n_rows):
  for j in range(n_cols):
    try: 
      feat= new_categorical[iteraz]
      grouped= df.select(feat, "Target").groupBy(feat).agg((F.mean('Target')).alias('Mean_Target'), F.count(feat).alias('N_samples'), F.stddev('Target').alias('Std_Target') 
                                                                         ).orderBy('N_samples', ascending= False)

      grouped_pandas= grouped.toPandas().iloc[:n_to_show].set_index(feat)
      grouped_pandas['N_samples']= (grouped_pandas['N_samples']/df.count()).cumsum()
      grouped_pandas.rename({'N_samples': 'Cum_%_samples'}, axis= 1, inplace= True)
      if len(grouped_pandas[grouped_pandas['Cum_%_samples'] < threshold_samples])> 2: # if the cumulative % of samples of the first two categories is higher than 70%
        grouped_pandas= grouped_pandas[grouped_pandas['Cum_%_samples'] < threshold_samples]
      else: #if the cumulative % of samples of the first two categories is higher than 70%
        grouped_pandas= grouped_pandas.iloc[:n_to_show]

      grouped_pandas['Mean_Target']= ((100*grouped_pandas['Mean_Target']/ proportion_class1)/ (1+ grouped_pandas['Mean_Target']/ proportion_class1))/100
      grouped_pandas[['Mean_Target', 'Std_Target']].plot(ax=ax[i][j],  kind= 'bar', title='{}'.format(feat))
      ax[i][j].set_ylim(0,1)  
      ax[i][j].set_yticks([0, 0.25, 0.5, 0.75, 1]) 
      
      iteraz+=1
    except:
      pass

plt.tight_layout(pad=1, h_pad=3.0) #w_pad=0.5
plt.savefig('Categorical_Plots.PNG')
display(fig)

In [78]:
series_counts.loc[new_categorical].sort_values(ascending= False).iloc[:]

Out[110]: industry_code 52
state_prev_res 51
occupation_code 47
country_self 43
country_mother 43
country_father 43
family_status 38
major_ind_code 24
education 17
major_occ_code 15
mig_chg_msa 10
hisp_origin 10
mig_move_reg 10
class_worker 9
mig_chg_reg 9
full_or_part_emp 8
family_summary 8
num_employees 7
marital_status 7
region_prev_res 6
tax_filer_stat 6
unemployment_reason 6
familiars_U18 5
citizenship 5
race 5
mig_prev_sunbelt 4
own_business 3
_c38 3
labor_union_member 3
mig_same 3
vet_question 3
hs_college 3
gender 2
survey_year 2
dtype: int64

In [79]:
new_numerical

Out[29]: ['age',
 'wage_per_hour',
 'capital_gains',
 'capital_losses',
 'stock_dividends',
 '_c24',
 'weeks_worked']

In [80]:
plt.figure(figsize=(9,6))
title_ticks_ecc('Correlation between features \n(icluding target)', '', '', new_fig= False)
sns.heatmap(num_pandas.corr(), cmap= 'coolwarm', annot= True) #[new_numerical + ['Target']]
display(plt.show())

In [81]:
df.select(new_numerical).describe().toPandas().set_index('summary').transpose()

summary,count,mean,stddev,min,max
age,192035,34.72070716275679,22.09622824805086,0,90
wage_per_hour,192035,56.630911031843155,276.85433826851903,0,9999
capital_gains,192035,437.4583279089749,4707.88652323138,0,99999
capital_losses,192035,37.66209284765798,272.5683732989905,0,4608
stock_dividends,192035,198.00374410914677,1990.2649211730638,0,99999
_c24,192035,1745.3538831392734,998.4988954083191,37.87,18656.3
weeks_worked,192035,23.625469315489365,24.427984606424875,0,52


##Train Test Split

In [83]:
df_train, df_test = df.randomSplit([0.8, 0.2], seed=0)
df_train.count()

Out[27]: 153720

#Feature Selection (basing on Training Set)

In [85]:
from pyspark.ml.stat import ChiSquareTest
from pyspark.mllib.linalg import Vectors,  Matrices  #occhio: non importare da pyspark.ml.linalg, sennò da errori
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics

# Convert everything to a LabeledPoint object, the main consumption data structure for most of mllib
#to_labeled_point = lambda x: LabeledPoint(x[0], Vectors.dense(x[1].toArray()))

In [86]:
contingency_table= df_train.crosstab("gender", "Target")
contingency_list= contingency_table.rdd.map(lambda x:[x[1],x[2]]).flatMap(lambda y: y).collect()
print(contingency_list)
mat= Matrices.dense(numRows=2, numCols=2, values= contingency_list)
goodnessOfFitTestResult= Statistics.chiSqTest(mat)
print("%s\n" % goodnessOfFitTestResult)

[78039, 2070, 66009, 7602]
Chi squared test summary:
method: pearson
degrees of freedom = 1 
statistic = 3901.045214386111 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

In [87]:
display(contingency_table)

gender_Target,0,1
female,78039,2070
male,66009,7602


In [88]:
display(df_train.crosstab( "Target", "familiars_U18",))

Target_familiars_U18,both parents present,father only present,mother only present,neither parent present,not in universe
1,2,0,0,0,9670
0,28404,1481,9896,1290,102977


In [89]:
print(Statistics.chiSqTest(Matrices.dense(5, 2, df_train.crosstab("familiars_U18", "Target").rdd.map(lambda x:[x[1],x[2]]).flatMap(lambda y: y).collect()))) #IMP: 5 righe, 2 colonne

Chi squared test summary:
method: pearson
degrees of freedom = 4 
statistic = 109086.96806230012 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

In [90]:
def compute_chi2(df, col1, col2, display_contingency= False):
  #### Creating Contingency Table in Relation to Target ####
  test = df.crosstab(col1, col2) 
  if display_contingency is True:
    display(test)
  vals = test.rdd.map(lambda x:[x[1],x[2]]).flatMap(lambda y: y).collect() #it is only the first two because it is the target
  n_rows= test.count()
  n_cols= len(test.columns)-1
  Matrix = Matrices.dense(n_rows, n_cols, vals)
  print("%s\n" % Statistics.chiSqTest(Matrix))
  return Statistics.chiSqTest(Matrix).pValue

compute_chi2(df_train, "gender", "Target")

Chi squared test summary:
method: pearson
degrees of freedom = 1 
statistic = 3901.045214386111 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

Out[34]: 0.0

In [91]:
compute_chi2(df_train, "mig_move_reg", "Target", display_contingency= True)  #Always Two columns, since the Target has two.

mig_move_reg_Target,0,1
different state in west,504,27
?,69999,4943
different state in midwest,414,20
nonmover,60810,4215
different county same state,2149,106
same county,7580,274
different state in south,739,53
different state in northeast,324,19
abroad,421,15
not in universe,1108,0


In [92]:
compute_chi2(df_train, "mig_move_reg", "Target")== 0.0

Chi squared test summary:
method: pearson
degrees of freedom = 9 
statistic = 112412.21331176787 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

Out[44]: True

### First attempt of Chi2 Test on all df_train

In [ ]:
run_cell= False

if run_cell:

    chi2_tests = {}
    failed= []
    final_categorical= new_categorical.copy()
    for col in new_categorical:
      if df_train.select(F.countDistinct(col)).rdd.map(lambda x: x[0]).collect()[0]== 1: #if it is a constant
        print('There is only one value of column "{}", thus there is surely not dipendence with any other column'.format(col))
        df_train= df_train.drop(col)
        df_test= df_test.drop(col)
        if col in final_categorical:
          final_categorical.remove(col)
        continue #go to the next step of the loop
      else:
        try:
          print('\n\t...Performing Chi2Test for "{}"...'.format(col))
          chi2_tests[col]= compute_chi2(df_train, col , 'Target')
        except:
          failed.append(col)
          pass

In [95]:
chi2_tests

Out[138]: {'class_worker': 0.0,
 'education': 0.0,
 'hs_college': 0.0,
 'marital_status': 0.0,
 'major_ind_code': 0.0,
 'major_occ_code': 0.0,
 'race': 0.0,
 'hisp_origin': 0.0,
 'gender': 0.0,
 'labor_union_member': 0.0,
 'unemployment_reason': 0.0,
 'full_or_part_emp': 0.0,
 'tax_filer_stat': 0.0,
 'region_prev_res': 0.0,
 'state_prev_res': 0.0,
 'family_summary': 0.0,
 'mig_chg_msa': 0.0,
 'mig_chg_reg': 0.0,
 'mig_move_reg': 0.0,
 'mig_same': 0.0,
 'mig_prev_sunbelt': 0.0,
 'familiars_U18': 0.0,
 'country_father': 0.0,
 'country_mother': 0.0,
 'country_self': 0.0,
 'citizenship': 0.0,
 'vet_question': 0.0,
 'industry_code': 0.0,
 'occupation_code': 0.0,
 'num_employees': 0.0,
 '_c38': 0.0,
 'own_business': 0.0,
 'survey_year': 5.804316381863295e-06}

##### Chi-Test on all features combinations

In [97]:
def generic_test_chi2(df, col1, col2):
  #### Creating Contingency Table ####
  test = df.crosstab(col1, col2) 
  relevant_columns= test.columns[1:] #skip the first
  vals = test.select(relevant_columns).rdd.map(lambda x:[x[i] for i in range(len(relevant_columns))]).flatMap(lambda y: y).collect()
  n_rows= test.count()
  n_cols= len(test.columns)-1
  Matrix = Matrices.dense(n_rows, n_cols, vals)
  print("%s\n" % Statistics.chiSqTest(Matrix))
  return Statistics.chiSqTest(Matrix).pValue

generic_test_chi2(df_train, "gender", "Target")

Chi squared test summary:
method: pearson
degrees of freedom = 1 
statistic = 3901.045214386111 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

Out[48]: 0.0

In [98]:
generic_test_chi2(df_train.sample(fraction= 0.005), 'family_status', 'family_summary')

Chi squared test summary:
method: pearson
degrees of freedom = 0 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

Out[50]: 1.0

In [99]:
from itertools import combinations
list(combinations(categorical_feats ,2))[:3]

Out[19]: [('class_worker', 'education'),
 ('class_worker', 'hs_college'),
 ('class_worker', 'marital_status')]

In [ ]:
combinations_cols = []
combined_cols =  list(combinations(categorical_feats,2))
failed_cols= []
for couple in combined_cols:
  print(couple)
  try:
    combinations_cols.append((couple, generic_(df_train, couple[0], couple[1])))
    print('Test Executed')
  except:
    failed_cols.append(failed_cols)
    pass

#### Now computing Chi-Test for smaller sample sizes

In [103]:
df_chi_test= df_train.sample(withReplacement= False, fraction=0.02, seed= 0)  #with 0.01 sono circa 3000 samples.
df_chi_test.count()

Out[35]: 3085

In [104]:
df_chi_test.persist()

Out[48]: 1502

In [ ]:
run_cell= False

if run_cell:

    chi2_tests_small = {}
    failed= []
    final_categorical= new_categorical.copy()
    for col in final_categorical:
      if df_chi_test.select(F.countDistinct(col)).rdd.map(lambda x: x[0]).collect()[0]== 1: #if it is a constant
        print('There is only one value of column "{}", thus there is surely not dipendence with any other column'.format(col))
        df_chi_test= df_chi_test.drop(col)
        #df_test= df_test.drop(col)
        if col in final_categorical:
          final_categorical.remove(col)
        continue #go to the next step of the loop
      else:
        try:
          print('\n\t...Performing Chi2Test for "{}"...'.format(col))
          chi2_tests_small[col]= compute_chi2(df_chi_test, col , 'Target')
        except:
          failed.append(col)
          pass

##### Considering only high frequency classes

In [109]:
chi_tests_grouped= {}
failed= []
final_categorical= new_categorical.copy()
for col in final_categorical:
  if df_chi_test.select(F.countDistinct(col)).rdd.map(lambda x: x[0]).collect()[0]== 1: #if it is a constant
    print('There is only one value of column "{}", thus there is surely not dipendence with any other column'.format(col))
    df_chi_test= df_chi_test.drop(col)
    #df_test= df_test.drop(col)
    if col in final_categorical:
      final_categorical.remove(col)
    continue #go to the next step of the loop
  else:
    try:
      grouped_df= df_chi_test.select(col, "Target").groupBy(col).agg((F.count(col)/df_chi_test.count()).alias('%_samples')).orderBy('%_samples', ascending= False)
      ######### Extracting the first classes which together occurr in 80% of data ##############
      pandas= grouped_df.toPandas()
      pandas['%_samples']= pandas['%_samples'].cumsum()
      pandas= pandas[pandas['%_samples']< 0.85].set_index(col)
      main_categories= list(pandas.index)
      ############### Filtering df by taking only those categories #####################
      filtered= df_chi_test.filter(df_chi_test[col].isin(main_categories))
      ############### Computing Chi2 Test on the filtered dataset ###################
      print('\n\t...Performing Chi2Test for "{}"...'.format(col))
      chi_tests_grouped[col]= compute_chi2(filtered, col , 'Target')
    except:
      failed.append(col)
      pass

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "class_worker"...
Chi squared test summary:
method: pearson
degrees of freedom = 0 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "education"...
Chi squared test summary:
method: pearson
degrees of freedom = 6 
statistic = 2130.7427848338193 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "hs_college"...
Chi squared test summary:
method: pearson
degrees of freedom = 1 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "marital_status"...
Chi squared test summary:
method: pearson
degrees of freedom = 0 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "major_ind_code"...
Chi squared test summary:
method: pearson
degrees of freedom = 9 
statistic = 906.885666387033 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "major_occ_code"...
Chi squared test summary:
method: pearson
degrees of freedom = 5 
statistic = 453.15966122408594 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "race"...
Chi squared test summary:
method: pearson
degrees of freedom = 0 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "hisp_origin"...
Chi squared test summary:
method: pearson
degrees of freedom = 1 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, plea

In [110]:
summary_test= pd.DataFrame(pd.Series(chi_tests_grouped), columns= ['p-value'])
under_5percent= summary_test[summary_test['p-value']< 0.05]
under_5percent

,p-value
education,0.000000e+00
major_ind_code,0.000000e+00
major_occ_code,0.000000e+00
full_or_part_emp,1.169065e-13
tax_filer_stat,0.000000e+00
family_status,0.000000e+00
family_summary,0.000000e+00
industry_code,0.000000e+00
occupation_code,0.000000e+00
num_employees,0.000000e+00


## Chi Squared Test with Bootstrap and Undersampling

In [112]:
n_bootstraps= 3
#thresh_total_frequency= 0.9
chi_tests_grouped= {}
final_categorical= new_categorical.copy()

for i in range(n_bootstraps):
  print('\n BOOTSTRAP ITERATION ', i+1)
  #### Creating new undersampled dataset ####
  df_chi_test= df_train.sample(withReplacement= False, fraction=0.02)  #with 0.02 they are about 3000 samples.
  for col in final_categorical:
    if df_chi_test.select(F.countDistinct(col)).rdd.map(lambda x: x[0]).collect()[0]== 1: #if it is a constant
      print('There is only one value of column "{}", thus there is surely not dipendence with any other column'.format(col))
      continue #go to the next step of the loop
    else:
      try:
        grouped_df= df_chi_test.select(col, "Target").groupBy(col).agg((F.count(col)/df_chi_test.count()).alias(
                                                                         '%_samples')).orderBy('%_samples', ascending= False)
        ######### Extracting the first classes which together occurr in 80% of data ##############
        pandas= grouped_df.toPandas()
        pandas['%_samples']= pandas['%_samples'].cumsum()
        pandas= pandas[pandas['%_samples']< 0.85].set_index(col)
        main_categories= list(pandas.index)
        ############### Filtering df by taking only those categories #####################
        filtered= df_chi_test.filter(df_chi_test[col].isin(main_categories))
        ############### Computing Chi2 Test on the filtered dataset ###################
        print('\n\t...Performing Chi2Test for "{}"...'.format(col))
        if col not in chi_tests_grouped.keys(): #if it is not in the dictionary already, I create a list with the relative p-Value
          chi_tests_grouped[col]= [compute_chi2(filtered, col , 'Target')]
        else: ##if it is in the dictionary, I append the p-value to the list
          chi_tests_grouped[col]= chi_tests_grouped[col]+ [compute_chi2(filtered, col , 'Target')]
      except:
        #failed.append(col)
        pass

BOOTSTRAP ITERATION 1
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "class_worker"...
Chi squared test summary:
method: pearson
degrees of freedom = 0 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "education"...
Chi squared test summary:
method: pearson
degrees of freedom = 6 
statistic = 2173.971765505997 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "hs_college"...
Chi squared test summary:
method: pearson
degrees of freedom = 1 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "marital_status"...
Chi squared test summary:
method: pearson
degrees of freedom = 0 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "major_ind_code"...
Chi squared test summary:
method: pearson
degrees of freedom = 8 
statistic = 2097.8668838952103 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "major_occ_code"...
Chi squared test summary:
method: pearson
degrees of freedom = 5 
statistic = 740.6957491604056 
pValue = 0.0 
Very strong presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "race"...
Chi squared test summary:
method: pearson
degrees of freedom = 0 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

	...Performing Chi2Test for "hisp_origin"...
Chi squared test summary:
method: pearson
degrees of freedom = 1 
statistic = 0.0 
pValue = 1.0 
No presumption against null hypothesis: the occurrence of the outcomes is statistically independent..

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stre

In [113]:
chi_tests_grouped

Out[43]: {'class_worker': [1.0, 1.0, 1.0],
 'education': [0.0, 0.0, 0.0],
 'hs_college': [1.0, 1.0, 1.0],
 'marital_status': [1.0, 0.0, 0.0],
 'major_ind_code': [0.0, 0.0, 0.0],
 'major_occ_code': [0.0, 0.0, 0.0],
 'race': [1.0, 1.0, 1.0],
 'hisp_origin': [1.0, 1.0, 1.0],
 'gender': [1.0, 1.0, 1.0],
 'labor_union_member': [1.0, 1.0, 1.0],
 'unemployment_reason': [1.0, 1.0, 1.0],
 'full_or_part_emp': [5.581003548194019e-09,
 1.865174681370263e-14,
 1.0447198661722723e-12],
 'tax_filer_stat': [0.0, 0.0, 0.0],
 'region_prev_res': [1.0, 1.0, 1.0],
 'state_prev_res': [1.0, 1.0, 1.0],
 'family_status': [0.0, 0.0, 0.0],
 'family_summary': [0.0, 0.0, 0.0],
 'mig_chg_msa': [1.0, 1.0, 1.0],
 'mig_chg_reg': [1.0, 1.0, 1.0],
 'mig_move_reg': [1.0, 1.0, 1.0],
 'mig_same': [1.0, 1.0, 1.0],
 'mig_prev_sunbelt': [1.0, 1.0, 1.0],
 'familiars_U18': [1.0, 1.0, 1.0],
 'country_father': [0.02525000677323941, 1.0, 1.0],
 'country_mother': [1.0, 1.0, 1.0],
 'country_self': [1.0, 1.0, 1.0],
 'citizenship': [1.0, 1.0, 1.0],
 'vet_question': [1.0, 1.0, 1.0],
 'industry_code': [0.0, 0.0, 0.0],
 'occupation_code': [0.0, 0.0],
 'num_employees': [0.0, 0.0, 0.0],
 '_c38': [1.0, 1.0, 1.0],
 'own_business': [1.0, 1.0, 1.0],
 'survey_year': [1.0, 1.0, 1.0]}

In [114]:
p_value_means= {}
for key in chi_tests_grouped:
  p_value_means[key]= np.array(chi_tests_grouped[key]).mean()
  
p_value_means

Out[46]: {'class_worker': 1.0,
 'education': 0.0,
 'hs_college': 1.0,
 'marital_status': 0.3333333333333333,
 'major_ind_code': 0.0,
 'major_occ_code': 0.0,
 'race': 1.0,
 'hisp_origin': 1.0,
 'gender': 1.0,
 'labor_union_member': 1.0,
 'unemployment_reason': 1.0,
 'full_or_part_emp': 1.8606889732690017e-09,
 'tax_filer_stat': 0.0,
 'region_prev_res': 1.0,
 'state_prev_res': 1.0,
 'family_status': 0.0,
 'family_summary': 0.0,
 'mig_chg_msa': 1.0,
 'mig_chg_reg': 1.0,
 'mig_move_reg': 1.0,
 'mig_same': 1.0,
 'mig_prev_sunbelt': 1.0,
 'familiars_U18': 1.0,
 'country_father': 0.6750833355910798,
 'country_mother': 1.0,
 'country_self': 1.0,
 'citizenship': 1.0,
 'vet_question': 1.0,
 'industry_code': 0.0,
 'occupation_code': 0.0,
 'num_employees': 0.0,
 '_c38': 1.0,
 'own_business': 1.0,
 'survey_year': 1.0}

In [115]:
avg_pvalues= pd.DataFrame(pd.Series(p_value_means), columns= ['avg_pvalue'])
cols_to_keep= avg_pvalues[avg_pvalues['avg_pvalue']< 0.05]
cols_to_keep

,avg_pvalue
education,0.000000e+00
major_ind_code,0.000000e+00
major_occ_code,0.000000e+00
full_or_part_emp,1.860689e-09
tax_filer_stat,0.000000e+00
family_status,0.000000e+00
family_summary,0.000000e+00
industry_code,0.000000e+00
occupation_code,0.000000e+00
num_employees,0.000000e+00


In [116]:
cols_to_keep= list(cols_to_keep.index)
cols_to_keep

Out[52]: ['education',
 'major_ind_code',
 'major_occ_code',
 'full_or_part_emp',
 'tax_filer_stat',
 'family_status',
 'family_summary',
 'industry_code',
 'occupation_code',
 'num_employees']

In [117]:
#df_chi2_tests = spark.createDataFrame(chi2_tests, ["feature_pair", "pvalue"])
#display(df_chi2_tests)

In [118]:
####### PERCENTAGE OF CLASSES WHOSE CUMULATIVE FREQUENCY IS 85% #########
n_classes_85_frequency= {}
for col in new_categorical:
  grouped_df= df_train.select(col, "Target").groupBy(col).agg((F.count(col)/df_train.count()).alias('%_samples')).orderBy('%_samples', ascending= False)
  ######### Extracting the first classes which together occurr in 80% of data ##############
  pandas= grouped_df.toPandas()
  pandas['%_samples']= pandas['%_samples'].cumsum()
  freq_85 = pandas[pandas['%_samples']< 0.85].set_index(col)
  n_classes_85_frequency[col]= len(freq_85)/len(pandas)

/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "
/databricks/python/lib/python3.7/site-packages/pyarrow/__i

In [119]:
#classes_85_frequency= pd.DataFrame(pd.Series(n_classes_85_frequency).sort_values(ascending= False), columns= ['%classes_cover_85%data'])
classes_85_frequency= classes_85_frequency.join(pd.DataFrame(series_counts), how='left')
classes_85_frequency.columns= ['%classes_cover_85%data',  'n_classes']
classes_85_frequency

,%classes_cover_85%data,n_classes
num_employees,0.571429,7
survey_year,0.500000,2
gender,0.500000,2
major_ind_code,0.416667,24
education,0.411765,17
major_occ_code,0.400000,15
occupation_code,0.382979,47
family_summary,0.375000,8
tax_filer_stat,0.333333,6
_c38,0.333333,3


In [120]:
######################################################################################################################################################################################

# Definitive Feature Selection

In [122]:
#### Taking the following columns ##########

#removed occupation code, industry_code and major_ind_code because correlated with num_employees. 
#_c24 because does not change basing on target
# Family summary because it is the simplified version of family_status.

if '_c24' in new_numerical:
  new_numerical.remove('_c24')
if  'num_employees' not in new_numerical:
  new_numerical.append('num_employees')
  
final_categorical=  ['education',
               'major_occ_code',
               'full_or_part_emp',
               'tax_filer_stat',
               'family_status']

cols_to_keep= final_categorical + new_numerical + ['Target']

cols_to_keep

Out[151]: ['education',
 'major_occ_code',
 'full_or_part_emp',
 'tax_filer_stat',
 'family_status',
 'age',
 'wage_per_hour',
 'capital_gains',
 'capital_losses',
 'stock_dividends',
 'weeks_worked',
 'num_employees',
 'Target']

In [123]:
new_df_train= df_train.select(cols_to_keep)
new_df_test= df_test.select(cols_to_keep)

"""for col in new_numerical:
  new_df_train= new_df_train.withColumn(col, new_df_train[col].cast('int'))
  new_df_test= new_df_test.withColumn(col, new_df_test[col].cast('int'))"""
  
new_df_train.printSchema()

root
-- education: string (nullable = true)
-- major_occ_code: string (nullable = true)
-- full_or_part_emp: string (nullable = true)
-- tax_filer_stat: string (nullable = true)
-- family_status: string (nullable = true)
-- age: integer (nullable = true)
-- wage_per_hour: double (nullable = true)
-- capital_gains: double (nullable = true)
-- capital_losses: double (nullable = true)
-- stock_dividends: double (nullable = true)
-- weeks_worked: double (nullable = true)
-- num_employees: double (nullable = true)
-- Target: integer (nullable = true)

In [124]:
display(new_df_train.head(3))

education,major_occ_code,full_or_part_emp,tax_filer_stat,family_status,age,wage_per_hour,capital_gains,capital_losses,stock_dividends,weeks_worked,num_employees,Target
children,not in universe,children or armed forces,nonfiler,child <18 never marr not in subfamily,0,0.0,0.0,0.0,0.0,0.0,0.0,0
children,not in universe,children or armed forces,nonfiler,child <18 never marr not in subfamily,0,0.0,0.0,0.0,0.0,0.0,0.0,0
children,not in universe,children or armed forces,nonfiler,child <18 never marr not in subfamily,0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [125]:
series_counts.loc[final_categorical].sort_values(ascending = False).sum() + len(new_numerical)

Out[154]: 91

## One Hot Encoding Strings

In [127]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer,  StandardScaler)

final_categorical

Out[155]: ['education',
 'major_occ_code',
 'full_or_part_emp',
 'tax_filer_stat',
 'family_status']

In [128]:
education_indexer = StringIndexer(inputCol='education',outputCol='educationIndex')
education_encoder = OneHotEncoder(inputCol='educationIndex',outputCol='educationVec')

occ_code_indexer = StringIndexer(inputCol='major_occ_code',outputCol='major_occIndex')
occ_code_encoder = OneHotEncoder(inputCol='major_occIndex',outputCol='major_occVec')

full_or_part_indexer = StringIndexer(inputCol='full_or_part_emp',outputCol='full_or_partIndex')
full_or_part_encoder = OneHotEncoder(inputCol='full_or_partIndex',outputCol='full_or_partVec')

tax_filer_indexer = StringIndexer(inputCol='tax_filer_stat',outputCol='tax_filerIndex')
tax_filer_encoder = OneHotEncoder(inputCol='tax_filerIndex',outputCol='tax_filerVec')

family_status_indexer = StringIndexer(inputCol='family_status',outputCol='family_statusIndex')
family_status_encoder = OneHotEncoder(inputCol='family_statusIndex',outputCol='family_statusVec')

target_indexer = StringIndexer(inputCol="Target", outputCol="Target")

In [129]:
"""encoded_categorical= ['educationVec', 'major_occVec', 'full_or_partVec', 'tax_filerVec', 'family_statusVec', 'family_summaryVec']
assembler = VectorAssembler(inputCols= new_numerical + encoded_categorical, outputCol='features')"""

Out[157]: "encoded_categorical= ['educationVec', 'major_occVec', 'full_or_partVec', 'tax_filerVec', 'family_statusVec', 'family_summaryVec']\nassembler = VectorAssembler(inputCols= new_numerical + encoded_categorical, outputCol='features')"

In [ ]:
num_assembler = VectorAssembler(inputCols = new_numerical, outputCol='num_features')
new_df_train = num_assembler.transform(new_df_train)

scaler = StandardScaler(inputCol="num_features", outputCol="scaledFeatures", withStd=True, withMean=True)
scalerModel = scaler.fit(new_df_train)
scaled_train = scalerModel.transform(new_df_train)

new_df_test= num_assembler.transform(new_df_test)
scaled_test = scalerModel.transform(new_df_test)

In [132]:
scaled_train.select('scaledFeatures').show(3)

+--------------------+
 scaledFeatures|
+--------------------+
[-1.5714434987339...|
[-1.5714434987339...|
[-1.5714434987339...|
+--------------------+
only showing top 3 rows

In [133]:
scaled_train.select('scaledFeatures').rdd.collect()[0]

Out[56]: Row(scaledFeatures=DenseVector([-1.5714, -0.2047, -0.0933, -0.1384, -0.0994, -0.9671, -0.8407]))

In [134]:
encoded_categorical= ['educationVec', 'major_occVec', 'full_or_partVec', 'tax_filerVec', 'family_statusVec', 'TargetIndex']
final_assembler = VectorAssembler(inputCols= encoded_categorical+ ['scaledFeatures'], outputCol='all_features')

In [135]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier
from pyspark.ml import Pipeline

rf_model = RandomForestClassifier(featuresCol='all_features', labelCol='Target', 
                                     numTrees=10, featureSubsetStrategy="sqrt",
                                     impurity='gini', maxDepth=4, )

pipeline = Pipeline(stages=[education_indexer,education_encoder, occ_code_indexer, occ_code_encoder, full_or_part_indexer, full_or_part_encoder,
                           tax_filer_indexer,tax_filer_encoder, family_status_indexer, family_status_encoder, target_indexer,
                           final_assembler, rf_model])

In [ ]:
################## Trial dataframes for saving time ##########################àà
trial_train= scaled_train.limit(8000)
trial_test= scaled_test.limit(2000)

#display(prova_train)

In [137]:
fit_model = pipeline.fit(trial_train)
results_train = fit_model.transform(trial_train)
results_test = fit_model.transform(trial_test)

In [138]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
ROC_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Target', metricName="areaUnderROC")

ACC_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Target', metricName="accuracy")

In [139]:
display(test_results.select('Target','prediction').limit(5))

Target,prediction
0,0.0
0,0.0
0,0.0
0,0.0
0,0.0


In [140]:
Train_accuracy = ACC_eval.evaluate(results_train)
Test_accuracy = ACC_eval.evaluate(results_test)
print('Train Accuracy is {}'.format(Train_accuracy))
print('Test Accuracy is {}'.format(Test_accuracy ))

Train Accuracy is 0.959
Test Accuracy is 0.916

In [141]:
Train_AUC = ROC_eval.evaluate(results_train)
Test_AUC = ROC_eval.evaluate(results_test)
print('Train Area Under ROC is {}'.format(Train_AUC))
print('Test Area Under ROC is {}'.format(Test_AUC))

Train Area Under ROC is 0.842
Test Area Under ROC is 0.728

In [142]:
############# GridSearch #################
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

paramGrid = ParamGridBuilder()\
    .addGrid(rf_model.numTrees, [50, 100, 500]) \
    .addGrid(rf_model.maxDepth, [3, 7, 15])\
    .addGrid(rf_model.impurity, ['gini', 'entropy'])\
    .build()

grid_search = TrainValidationSplit(estimator=rf_model,
                           estimatorParamMaps=paramGrid,
                           evaluator= BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Target'),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.8)

grid_model = grid_search.fit(scaled_train)
train_predictions= model.transform(scaled_train)
test_predictions= model.transform(scaled_test)

##..Fitting all the training Set.. (not done for lack of time and computational power)

In [144]:
fit_model = pipeline.fit(scaled_train)
results = fit_model.transform(scaled_test)

In [145]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Target')
results.select('Target','prediction').sample(fraction= 0.1).show()

+------+----------+
Target|prediction|
+------+----------+
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 1| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
 0| 0.0|
+------+----------+
only showing top 20 rows

In [146]:
AUC = my_eval.evaluate(results)
AUC